<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE60499](https://ndcbe.github.io/CBE60499);
content is available [on Github](git@github.com:ndcbe/CBE60499.git).*


<!--NAVIGATION-->
< [4.4 Constraint Qualifications](https://ndcbe.github.io/CBE60499/04.04-Constraint-Qualifications.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [4.6 NLP Diagnostics with Degeneracy Hunter](https://ndcbe.github.io/CBE60499/04.06-NLP-Diagnostics.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/04.05-Second-Order.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/04.05-Second-Order.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 4.5 Second Order Optimality Conditions

In [1]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.download_figures(['cone1.png','cone2.png'])

## 4.5.1 Helpful Cones

![picture](./figures/cone1.png)

![picture](./figures/cone2.png)

## 4.5.2 Second Order Necessary Conditions

![picture](./figures/thm-4-17.png)

## 4.5.3 Second Order Sufficient Conditions

![picture](./figures/thm-4-18.png)

## 4.5.4 Reduced Hessian

![picture](./figures/def-4-19.png)

![picture](./figures/projected-reduced-Hessian1.png)

![picture](./figures/projected-reduced-Hessian2.png)

## 4.5.5 Example

![picture](./figures/ex-4-20.png)

See errata here: http://numero.cheme.cmu.edu/content/errata2.pdf

![picture](./figures/errata_83.png)

### 4.5.5.1 Calculation with numpy

In [2]:
import numpy as np

# Define the Hessian of the Lagrange function
hessian_L = np.array([[2.3093, 0.4315], [0.4315, 3.2021]])
print(hessian_L)

[[2.3093 0.4315]
 [0.4315 3.2021]]


In [3]:
# Define the gradiant of the constraints
A = np.reshape(np.array([-2.2245, -1.7981]),(2,1))

In [4]:
print(A)

[[-2.2245]
 [-1.7981]]


Calculate the complete QR factorization.

$$
A = Q \times R
$$

In [5]:
# Calculate the COMPLETE QR factorization
Q, R = np.linalg.qr(A, mode='complete')

In [6]:
print(Q)

[[-0.77770385 -0.62863083]
 [-0.62863083  0.77770385]]


In [7]:
print(R)

[[2.86034331]
 [0.        ]]


In [8]:
print(Q @ R)

[[-2.2245]
 [-1.7981]]


The second column of $Q$ is the null space of $A$. How do we know it is the null space? It corresponds to the second element of $R$ which is 0.

Finally, we can calculate the reduced Hessian:

In [9]:
# Calculate the reduced Hessian
print("Reduced Hessian w.r.t. null space of constraints:")
Q[:,1].T @ hessian_L @ Q[:,1]

Reduced Hessian w.r.t. null space of constraints:


2.4273753426093774

### 4.5.5.2 Calculate with Pyomo

#### 4.5.5.2.1 Define and solve the model

In [10]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()

# Define variables
m.x1 = pyo.Var(initialize=1)
m.x2 = pyo.Var(initialize=1)

# Define constraints
# Note: changing this to an equality constraint
# changes the reduced_hessian
m.con1 = pyo.Constraint(expr=4 - m.x1*m.x2 <= 0 )

# Define objective
m.obj = pyo.Objective(expr=m.x1**2 - 4*m.x1 + 1.5*m.x2**2 - 7*m.x2 + m.x1*m.x2 + 9 - pyo.log(m.x1) - pyo.log(m.x2))

# Obtain dual solutions from first solve and send to warm start
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# Specify Ipopt as the solver and solve
opt = pyo.SolverFactory('ipopt')
opt.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of ineq

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 2, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.04768204689025879}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

#### 4.5.5.2.2 Extract the dual variable for the constraint

In [11]:
m.dual[m.con1]

-0.5684977067847848

This is negative because of a sign convention in Pyomo. Notice it matches the book (expect the sign).

#### 4.5.5.2.3 Extract the reduced Hessian

In [12]:
# Warning: this Pyomo feature is experimental

from pyomo.contrib.interior_point.inverse_reduced_hessian import inv_reduced_hessian_barrier
# https://github.com/Pyomo/pyomo/blob/main/pyomo/contrib/interior_point/inverse_reduced_hessian.py

Compute the reduced Hessian with respect to $x_1$

In [13]:
solve_result, inv_red_hes = inv_reduced_hessian_barrier(
                        m,
                        independent_variables= [m.x1], # Warning: these variables cannot be at their bounds
                        tee=True)

print("\nReduced Hessian w.r.t. x1:")
print(np.linalg.inv(inv_red_hes))

Ipopt 3.13.2: bound_relax_factor=0
honor_original_bounds=no


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints..

Compute the reduced Hessian with respect to $x_2$

In [14]:
solve_result, inv_red_hes = inv_reduced_hessian_barrier(
                        m,
                        independent_variables= [m.x2], # Warning: these variables cannot be at their bounds
                        tee=True)

print("\nReduced Hessian w.r.t. x2:")
print(np.linalg.inv(inv_red_hes))

Ipopt 3.13.2: bound_relax_factor=0
honor_original_bounds=no


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints..

**Take away message**: The reduced Hessian relative to all three bases (null space, $[1, 0]$, and $[0, 1]$) is positive definite.

<!--NAVIGATION-->
< [4.4 Constraint Qualifications](https://ndcbe.github.io/CBE60499/04.04-Constraint-Qualifications.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [4.6 NLP Diagnostics with Degeneracy Hunter](https://ndcbe.github.io/CBE60499/04.06-NLP-Diagnostics.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/04.05-Second-Order.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/04.05-Second-Order.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>